<a href="https://colab.research.google.com/github/LorenzoAgnolucci/BERT_for_ABSA/blob/master/BERT_for_ABSA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive




---


# BERT-pair


---



In [ ]:
#@title Choose a dataset and a task { run: "auto", display-mode: "form" }
base_dir = "/gdrive/MyDrive/Machine_Learning" #@param {type:"string"}
dataset_type = "sentihood" #@param ["sentihood", "semeval2014"]
task = "NLI_M" #@param ["QA_M", "NLI_M", "QA_B", "NLI_B"]

In [ ]:
import pandas as pd
import random


if dataset_type == "sentihood":
    id2label = {0: "None", 1: "Positive", 2: "Negative"}
    label2id = {"None": 0, "Positive": 1, "Negative": 2}
elif dataset_type == "semeval2014":
    id2label = {0: "positive", 1: "neutral", 2: "negative", 3: "conflict", 4: "none"}
    label2id = {"positive": 0, "neutral" : 1, "negative" : 2, "conflict": 3, "none": 4}

if task.endswith("B"):
    num_classes = 2
else:
    if dataset_type == "sentihood":
        num_classes = 3
    elif dataset_type == "semeval2014":
        num_classes = 5


def get_dataset(path):
    original_sentences = []
    auxiliary_sentences = []
    labels = []
    data = pd.read_csv(path, header=0, sep="\t").values.tolist()
    for row in data:
        original_sentences.append(row[1])
        auxiliary_sentences.append(row[2])
        labels.append(row[3])
    return original_sentences, auxiliary_sentences, labels



train_original_sentences, train_auxiliary_sentences, train_labels = get_dataset(f"{base_dir}/data/{dataset_type}/BERT-pair/train_{task}.csv")
if dataset_type == "sentihood":
    val_original_sentences, val_auxiliary_sentences, val_labels = get_dataset(f"{base_dir}/data/{dataset_type}/BERT-pair/dev_{task}.csv")
elif dataset_type == "semeval2014":
    val_original_sentences, val_auxiliary_sentences, val_labels = get_dataset(f"{base_dir}/data/{dataset_type}/BERT-pair/test_{task}.csv")
test_original_sentences, test_auxiliary_sentences, test_labels = get_dataset(f"{base_dir}/data/{dataset_type}/BERT-pair/test_{task}.csv")

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = tokenizer(train_original_sentences, train_auxiliary_sentences, truncation=True, padding=True)
val_encodings = tokenizer(val_original_sentences, val_auxiliary_sentences, truncation=True, padding=True)
test_encodings = tokenizer(test_original_sentences, test_auxiliary_sentences, truncation=True, padding=True)

In [ ]:
import torch

class ABSA_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ABSA_Dataset(train_encodings, train_labels)
val_dataset = ABSA_Dataset(val_encodings, val_labels)
test_dataset = ABSA_Dataset(test_encodings, test_labels)

In [ ]:
import sys
if base_dir not in sys.path:
    sys.path.insert(0, f'{base_dir}/')
import numpy as np
from scipy.special import softmax
import evaluation


def get_test_labels(data_dir, dataset_type):
    original_sentences = []
    auxiliary_sentences = []
    labels = []
    data = pd.read_csv(f"{data_dir}/{dataset_type}/BERT-pair/test_NLI_M.csv", header=0, sep="\t").values.tolist()
    for row in data:
        labels.append(row[3])
    return labels


def get_predictions(data, task, dataset_type):
    predicted_labels = []
    scores = []
    if task.endswith("B"):
        if dataset_type == "sentihood":
            if task.endswith("B"):
                count_aspect_rows = 0
                current_aspect_scores = []
                for row in data:
                    current_aspect_scores.append(row[2])
                    count_aspect_rows += 1
                    if count_aspect_rows % 3 == 0:
                        sum_current_aspect_scores = np.sum(current_aspect_scores)
                        current_aspect_scores = [score / sum_current_aspect_scores for score in current_aspect_scores]
                        scores.append(current_aspect_scores)
                        predicted_labels.append(np.argmax(current_aspect_scores))
                        current_aspect_scores = []
        elif dataset_type == "semeval2014":
            if task.endswith("B"):
                count_aspect_rows = 0
                current_aspect_scores = []
                for row in data:
                    current_aspect_scores.append(row[2])
                    count_aspect_rows += 1
                    if count_aspect_rows % 5 == 0:
                        sum_current_aspect_scores = np.sum(current_aspect_scores)
                        current_aspect_scores = [score / sum_current_aspect_scores for score in current_aspect_scores]
                        scores.append(current_aspect_scores)
                        predicted_labels.append(np.argmax(current_aspect_scores))
                        current_aspect_scores = []
    return predicted_labels, scores


if dataset_type == "sentihood":
    def compute_metrics(predictions):
        scores = [softmax(prediction) for prediction in predictions[0]]
        predicted_labels = [np.argmax(x) for x in scores]
        if task.endswith("B"):
            data = np.insert(scores, 0, predicted_labels, axis=1)
            predicted_labels, scores = get_predictions(data, task, dataset_type)
        test_labels = get_test_labels(f"{base_dir}/data", dataset_type)
        metrics = {}
        metrics["strict_acc"] = evaluation.compute_sentihood_aspect_strict_accuracy(test_labels, predicted_labels)
        metrics["F1"] = evaluation.compute_sentihood_aspect_macro_F1(test_labels, predicted_labels)
        metrics["aspect_AUC"] = evaluation.compute_sentihood_aspect_macro_AUC(test_labels, scores)
        sentiment_macro_AUC, sentiment_accuracy = evaluation.compute_sentihood_sentiment_classification_metrics(test_labels, scores)
        metrics["sentiment_acc"] = sentiment_accuracy
        metrics["sentiment_AUC"] = sentiment_macro_AUC
        return metrics

elif dataset_type == "semeval2014":
    def compute_metrics(predictions):
        scores = [softmax(prediction) for prediction in predictions[0]]
        predicted_labels = [np.argmax(x) for x in scores]
        if task.endswith("B"):
            data = np.insert(scores, 0, predicted_labels, axis=1)
            predicted_labels, scores = get_predictions(data, task, dataset_type)
        test_labels = get_test_labels(f"{base_dir}/data", dataset_type)
        metrics = {}
        p, r, f1 = evaluation.compute_semeval_PRF(test_labels, predicted_labels)
        metrics["P"] = p
        metrics["R"] = r
        metrics["F1"] = f1
        metrics["4-way"] = evaluation.compute_semeval_accuracy(test_labels, predicted_labels, scores, 4)
        metrics["3-way"] = evaluation.compute_semeval_accuracy(test_labels, predicted_labels, scores, 3)
        metrics["binary"] = evaluation.compute_semeval_accuracy(test_labels, predicted_labels, scores, 2)
        return metrics

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, BertConfig

from transformers import logging
logging.set_verbosity_debug()


epochs = 4
batch_size = 24
num_steps = len(train_dataset) * epochs // batch_size
warmup_steps = num_steps // 10  # 10% of the training steps
save_steps = num_steps // epochs    # Save a checkpoint at the end of each epoch


training_args = TrainingArguments(
    output_dir = f'{base_dir}/models/{dataset_type}/BERT-pair/{task}/',          
    num_train_epochs = epochs,              
    per_device_train_batch_size = batch_size,  
    per_device_eval_batch_size = batch_size,   
    warmup_steps = warmup_steps,   
    weight_decay = 0.01,               
    logging_dir = f'{base_dir}/logs/{dataset_type}/BERT-pair/{task}/',            
    logging_steps = 10,
    evaluation_strategy = 'epoch',
    learning_rate = 2e-5,
    save_steps = save_steps
)

config = BertConfig.from_pretrained(
    'bert-base-uncased',
    architectures = ['BertForSequenceClassification'],
    hidden_size = 768,
    num_hidden_layers = 12,
    num_attention_heads = 12,
    hidden_dropout_prob = 0.1,
    num_labels = num_classes
)    

load_finetuned_model = True
if not load_finetuned_model:
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)

    trainer = Trainer(
        model=model,                         
        args=training_args,                  
        train_dataset=train_dataset,         
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics             
    )
    trainer.train()

    model.save_pretrained(f"{base_dir}/models/{dataset_type}/BERT-pair/{task}/last_step")

else:
    model = BertForSequenceClassification.from_pretrained(f"{base_dir}/models/{dataset_type}/BERT-pair/{task}/last_step")

    trainer = Trainer(
        model=model,                         
        args=training_args,                  
        train_dataset=train_dataset,         
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics             
    )


PyTorch: setting up devices
loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.637c6035640bacb831febcc2b7f7bee0a96f9b30c2d7e9ef84082d9f252f3170
Model config BertConfig {
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.2.2",
  "type_v

In [ ]:
evaluation_result = trainer.evaluate(test_dataset)
print(evaluation_result)

***** Running Evaluation *****
  Num examples = 7516
  Batch size = 24


{'eval_loss': 0.221498042345047, 'eval_strict_acc': 0.7796700372538584, 'eval_F1': 0.8704235253645549, 'eval_aspect_AUC': 0.9722000837833961, 'eval_sentiment_acc': 0.9268092105263158, 'eval_sentiment_AUC': 0.9579929215675353, 'eval_combined_loss': 0.008285065805207338, 'eval_runtime': 94.8867, 'eval_samples_per_second': 79.21}


In [ ]:
import pandas as pd


results = trainer.predict(test_dataset)

scores = [softmax(prediction) for prediction in results.predictions]
predicted_labels = [np.argmax(x) for x in scores]

***** Running Prediction *****
  Num examples = 20000
  Batch size = 24


In [ ]:
csv_output = np.insert(scores, 0, predicted_labels, axis=1)
df = pd.DataFrame(csv_output)
df[0] = df[0].astype("int")
if task.endswith("B"):
    header = ["predicted_label", "no", "yes"]
else:
    header = ["predicted_label"]
    for label in label2id.keys():
        header.append(label)
df.to_csv(f"{base_dir}/results/{dataset_type}/BERT-pair/{task}.csv", index=False, header=header)

In [ ]:
import sys
if base_dir not in sys.path:
    sys.path.insert(0, f'{base_dir}/')
import evaluation
evaluation.main(task, dataset_type, f"{base_dir}/data", f"{base_dir}/results")

QA_M Sentihood aspect strict accuracy: 0.7653006918573709
QA_M Sentihood aspect macro F1: 0.8766424576478523
QA_M Sentihood aspect macro AUC: 0.965071463276167
QA_M Sentihood sentiment accuracy: 0.9259868421052632
QA_M Sentihood sentiment macro AUC: 0.9615643417277243




---


# BERT-single


---



In [ ]:
#@title Choose a dataset and a task { run: "auto", display-mode: "form" }
base_dir = "/gdrive/MyDrive/Machine_Learning" #@param {type:"string"}
dataset_type = "semeval2014" #@param ["sentihood", "semeval2014"]
task = "single" #@param ["single"]

In [ ]:
import pandas as pd
import random


if dataset_type == "sentihood":
    id2label = {0: "None", 1: "Positive", 2: "Negative"}
    label2id = {"None": 0, "Positive": 1, "Negative": 2}
elif dataset_type == "semeval2014":
    id2label = {0: "positive", 1: "neutral", 2: "negative", 3: "conflict", 4: "none"}
    label2id = {"positive": 0, "neutral" : 1, "negative" : 2, "conflict": 3, "none": 4}

if dataset_type == "sentihood":
    num_classes = 3
    locations = ["location_1_", "location_2_"]
    aspects = ["general", "price", "safety", "transit location"]
elif dataset_type == "semeval2014":
    num_classes = 5
    locations = [""]
    aspects = ["ambience", "anecdotes", "food", "price", "service"]


def get_dataset(path):
    original_sentences = []
    labels = []
    data = pd.read_csv(path, header=0, sep="\t").values.tolist()
    for row in data:
        original_sentences.append(row[1])
        labels.append(row[3])
    return original_sentences, labels


train_original_sentences = {}
train_labels = {}
val_original_sentences = {}
val_labels = {}
test_original_sentences = {}
test_labels = {}

for location in locations:
    train_original_sentences[location] = {}
    train_labels[location] = {}
    val_original_sentences[location] = {}
    val_labels[location] = {}
    test_original_sentences[location] = {}
    test_labels[location] = {}
    for aspect in aspects:
        train_original_sentences[location][aspect], train_labels[location][aspect] = get_dataset(f"{base_dir}/data/{dataset_type}/BERT-single/{location}{aspect}/train.csv")
        if dataset_type == "sentihood":
            val_original_sentences[location][aspect], val_labels[location][aspect] = get_dataset(f"{base_dir}/data/{dataset_type}/BERT-single/{location}{aspect}/dev.csv")
        elif dataset_type == "semeval2014":
            val_original_sentences[location][aspect], val_labels[location][aspect] = get_dataset(f"{base_dir}/data/{dataset_type}/BERT-single/{location}{aspect}/test.csv")
        test_original_sentences[location][aspect], test_labels[location][aspect] = get_dataset(f"{base_dir}/data/{dataset_type}/BERT-single/{location}{aspect}/test.csv")

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

train_encodings = {}
val_encodings = {}
test_encodings = {}
for location in locations:
    train_encodings[location] = {}
    val_encodings[location] = {}
    test_encodings[location] = {}
    for aspect in aspects:
        train_encodings[location][aspect] = tokenizer(train_original_sentences[location][aspect], truncation=True, padding=True)
        val_encodings[location][aspect] = tokenizer(val_original_sentences[location][aspect], truncation=True, padding=True)
        test_encodings[location][aspect] = tokenizer(test_original_sentences[location][aspect], truncation=True, padding=True)

In [ ]:
import torch

class ABSA_Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


train_dataset = {}
val_dataset = {}
test_dataset = {}
for location in locations:
    train_dataset[location] = {}
    val_dataset[location] = {}
    test_dataset[location] = {}
    for aspect in aspects:
        train_dataset[location][aspect] = ABSA_Dataset(train_encodings[location][aspect], train_labels[location][aspect])
        val_dataset[location][aspect] = ABSA_Dataset(val_encodings[location][aspect], val_labels[location][aspect])
        test_dataset[location][aspect] = ABSA_Dataset(test_encodings[location][aspect], test_labels[location][aspect])

In [ ]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments, BertConfig
from transformers import logging
import gc
import pandas as pd
import numpy as np
from scipy.special import softmax


logging.set_verbosity_debug()


epochs = 4
batch_size = 24

header = ["predicted_label"]
for label in label2id.keys():
    header.append(label)

config = BertConfig.from_pretrained(
        'bert-base-uncased',
        architectures = ['BertForSequenceClassification'],
        hidden_size = 768,
        num_hidden_layers = 12,
        num_attention_heads = 12,
        hidden_dropout_prob = 0.1,
        num_labels = num_classes
    )    

for location in locations:
    for aspect in aspects:
        num_steps = len(train_dataset[location][aspect]) * epochs // batch_size
        warmup_steps = num_steps // 10  # 10% of the training steps
        save_steps = num_steps // epochs    # Save a checkpoint at the end of each epoch


        training_args = TrainingArguments(
            output_dir = f'{base_dir}/models/{dataset_type}/BERT-single/{location}{aspect}/',          
            num_train_epochs = epochs,              
            per_device_train_batch_size = batch_size,  
            per_device_eval_batch_size = batch_size,   
            warmup_steps = warmup_steps,   
            weight_decay = 0.01,               
            logging_dir = f'{base_dir}/logs/{dataset_type}/BERT-single/{location}{aspect}/',            
            logging_steps = 10,
            evaluation_strategy = 'epoch',
            learning_rate = 2e-5,
            save_steps = save_steps,
            seed=21
        )

        model = BertForSequenceClassification.from_pretrained('bert-base-uncased', config=config)

        trainer = Trainer(
            model=model,                         
            args=training_args,                  
            train_dataset=train_dataset[location][aspect],         
            eval_dataset=val_dataset[location][aspect]             
        )

        trainer.train()

        model.save_pretrained(f"{base_dir}/models/{dataset_type}/BERT-single/{location}{aspect}/last_step")

        results = trainer.predict(test_dataset[location][aspect])

        scores = [softmax(prediction) for prediction in results.predictions]
        predicted_labels = [np.argmax(x) for x in scores]

        csv_output = np.insert(scores, 0, predicted_labels, axis=1)
        df = pd.DataFrame(csv_output)
        df[0] = df[0].astype("int")
        df.to_csv(f"{base_dir}/results/{dataset_type}/BERT-single/{location}{aspect}.csv", index=False, header=header)

        del training_args
        del model
        del trainer
        del results
        del scores
        del predicted_labels
        del csv_output
        del df
        gc.collect()


loading configuration file https://huggingface.co/bert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/3c61d016573b14f7f008c02c4e51a366c67ab274726fe2910691e2a761acf43e.637c6035640bacb831febcc2b7f7bee0a96f9b30c2d7e9ef84082d9f252f3170
Model config BertConfig {
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolut

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,0.359600,0.273954,6.914300,115.702000
2,0.278700,0.264778,6.868700,116.470000
3,0.164400,0.250466,6.890900,116.095000
4,0.108200,0.249512,6.873700,116.386000


Saving model checkpoint to /gdrive/MyDrive/Machine_Learning/models/semeval2014/BERT-single/ambience/checkpoint-126
Configuration saved in /gdrive/MyDrive/Machine_Learning/models/semeval2014/BERT-single/ambience/checkpoint-126/config.json
Model weights saved in /gdrive/MyDrive/Machine_Learning/models/semeval2014/BERT-single/ambience/checkpoint-126/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 800
  Batch size = 24
Saving model checkpoint to /gdrive/MyDrive/Machine_Learning/models/semeval2014/BERT-single/ambience/checkpoint-252
Configuration saved in /gdrive/MyDrive/Machine_Learning/models/semeval2014/BERT-single/ambience/checkpoint-252/config.json
Model weights saved in /gdrive/MyDrive/Machine_Learning/models/semeval2014/BERT-single/ambience/checkpoint-252/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 800
  Batch size = 24
Saving model checkpoint to /gdrive/MyDrive/Machine_Learning/models/semeval2014/BERT-single/ambience/checkpoint-378
Configurati

PyTorch: setting up devices
loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are ini

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,0.726500,0.609629,6.876800,116.333000
2,0.568700,0.505199,6.891600,116.084000
3,0.444900,0.484813,6.894300,116.037000
4,0.227600,0.484368,6.881000,116.262000


...

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,0.227200,0.156943,6.889000,116.127000
2,0.171400,0.125422,6.879500,116.288000
3,0.128600,0.094198,6.878600,116.303000
4,0.069900,0.090618,6.881800,116.248000


Saving model checkpoint to /gdrive/MyDrive/Machine_Learning/models/semeval2014/BERT-single/price/checkpoint-126
Configuration saved in /gdrive/MyDrive/Machine_Learning/models/semeval2014/BERT-single/price/checkpoint-126/config.json
Model weights saved in /gdrive/MyDrive/Machine_Learning/models/semeval2014/BERT-single/price/checkpoint-126/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 800
  Batch size = 24
Saving model checkpoint to /gdrive/MyDrive/Machine_Learning/models/semeval2014/BERT-single/price/checkpoint-252
Configuration saved in /gdrive/MyDrive/Machine_Learning/models/semeval2014/BERT-single/price/checkpoint-252/config.json
Model weights saved in /gdrive/MyDrive/Machine_Learning/models/semeval2014/BERT-single/price/checkpoint-252/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 800
  Batch size = 24
Saving model checkpoint to /gdrive/MyDrive/Machine_Learning/models/semeval2014/BERT-single/price/checkpoint-378
Configuration saved in /gdrive/M

PyTorch: setting up devices
loading weights file https://huggingface.co/bert-base-uncased/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/a8041bf617d7f94ea26d15e218abd04afc2004805632abc0ed2066aa16d50d04.faf6ea826ae9c5867d12b22257f9877e6b8367890837bd60f7c54a29633f7f2f
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are ini

Epoch,Training Loss,Validation Loss,Runtime,Samples Per Second
1,0.340500,0.274201,6.882400,116.239000
2,0.242100,0.170125,6.880100,116.277000
3,0.152500,0.176227,6.884200,116.208000
4,0.093700,0.184986,6.884900,116.197000


Saving model checkpoint to /gdrive/MyDrive/Machine_Learning/models/semeval2014/BERT-single/service/checkpoint-126
Configuration saved in /gdrive/MyDrive/Machine_Learning/models/semeval2014/BERT-single/service/checkpoint-126/config.json
Model weights saved in /gdrive/MyDrive/Machine_Learning/models/semeval2014/BERT-single/service/checkpoint-126/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 800
  Batch size = 24
Saving model checkpoint to /gdrive/MyDrive/Machine_Learning/models/semeval2014/BERT-single/service/checkpoint-252
Configuration saved in /gdrive/MyDrive/Machine_Learning/models/semeval2014/BERT-single/service/checkpoint-252/config.json
Model weights saved in /gdrive/MyDrive/Machine_Learning/models/semeval2014/BERT-single/service/checkpoint-252/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 800
  Batch size = 24
Saving model checkpoint to /gdrive/MyDrive/Machine_Learning/models/semeval2014/BERT-single/service/checkpoint-378
Configuration save

In [ ]:
import sys
if base_dir not in sys.path:
    sys.path.insert(0, f'{base_dir}/')
import evaluation


evaluation.main(task, dataset_type, f"{base_dir}/data", f"{base_dir}/results")

single Semeval aspect precision: 0.9323076923076923
single Semeval aspect recall: 0.8868292682926829
single Semeval aspect micro F1: 0.9090000000000001
single Semeval 4-classes accuracy: 0.824390243902439
single Semeval 3-classes accuracy: 0.8653648509763617
single Semeval 2-classes accuracy: 0.9271899886234357


# Paper Code

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
!git clone https://github.com/HSLCY/ABSA-BERT-pair.git
%cd ABSA-BERT-pair/
%cd generate/
!bash make.sh sentihood
!bash make.sh semeval

Cloning into 'ABSA-BERT-pair'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 36 (delta 0), reused 1 (delta 0), pack-reused 33
Unpacking objects: 100% (36/36), done.
/content/ABSA-BERT-pair
/content/ABSA-BERT-pair/generate
len(train) =  15008
len(val) =  3748
len(test) =  7516
len(train) =  15008
len(val) =  3748
len(test) =  7516
len(train) =  15008
len(val) =  3748
len(test) =  7516
len(train) =  15008
len(val) =  3748
len(test) =  7516
Finished!
Finished!


In [ ]:
%cd ../
!wget https://storage.googleapis.com/bert_models/2020_02_20/uncased_L-12_H-768_A-12.zip
!unzip -d uncased_L-12_H-768_A-12 uncased_L-12_H-768_A-12.zip

/content/ABSA-BERT-pair
--2021-01-18 10:55:53--  https://storage.googleapis.com/bert_models/2020_02_20/uncased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.5.240, 172.217.7.144, 172.217.7.176, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.5.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 408102251 (389M) [application/zip]
Saving to: ‘uncased_L-12_H-768_A-12.zip’

uncased_L-12_H-768_ 100%[===================>] 389.20M   197MB/s    in 2.0s    

2021-01-18 10:55:55 (197 MB/s) - ‘uncased_L-12_H-768_A-12.zip’ saved [408102251/408102251]

Archive:  uncased_L-12_H-768_A-12.zip
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-12_H-768_A-12/bert_config.json  
  inflating: uncased_L-12_H-768_A-12/vocab.txt  
  inflating: uncased_L-12_H-768_A-12/bert_model.ckpt.index  


In [ ]:
!python convert_tf_checkpoint_to_pytorch.py \
--tf_checkpoint_path uncased_L-12_H-768_A-12/bert_model.ckpt \
--bert_config_file uncased_L-12_H-768_A-12/bert_config.json \
--pytorch_dump_path uncased_L-12_H-768_A-12/pytorch_model.bin

2021-01-18 10:56:02.280821: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
Converting TensorFlow checkpoint from uncased_L-12_H-768_A-12/bert_model.ckpt
Loading bert/embeddings/LayerNorm/beta with shape [768]
Numpy array shape (768,)
Loading bert/embeddings/LayerNorm/gamma with shape [768]
Numpy array shape (768,)
Loading bert/embeddings/position_embeddings with shape [512, 768]
Numpy array shape (512, 768)
Loading bert/embeddings/token_type_embeddings with shape [2, 768]
Numpy array shape (2, 768)
Loading bert/embeddings/word_embeddings with shape [30522, 768]
Numpy array shape (30522, 768)
Loading bert/encoder/layer_0/attention/output/LayerNorm/beta with shape [768]
Numpy array shape (768,)
Loading bert/encoder/layer_0/attention/output/LayerNorm/gamma with shape [768]
Numpy array shape (768,)
Loading bert/encoder/layer_0/attention/output/dense/bias with shape [768]
Numpy array shape (768,)
Loading bert/encoder/lay

In [ ]:
!CUDA_VISIBLE_DEVICES=0 python run_classifier_TABSA.py \
--task_name sentihood_QA_M \
--data_dir data/sentihood/bert-pair/ \
--vocab_file uncased_L-12_H-768_A-12/vocab.txt \
--bert_config_file uncased_L-12_H-768_A-12/bert_config.json \
--init_checkpoint uncased_L-12_H-768_A-12/pytorch_model.bin \
--eval_test \
--do_lower_case \
--max_seq_length 512 \
--train_batch_size 8 \
--learning_rate 2e-5 \
--num_train_epochs 4.0 \
--output_dir results/ \
--seed 42

Output streaming troncato alle ultime 5000 righe.
Iteration:  36% 667/1876 [10:42<19:27,  1.04it/s]
Iteration:  36% 668/1876 [10:43<19:22,  1.04it/s]
Iteration:  36% 669/1876 [10:44<19:18,  1.04it/s]
Iteration:  36% 670/1876 [10:44<19:14,  1.04it/s]
Iteration:  36% 671/1876 [10:45<19:11,  1.05it/s]
Iteration:  36% 672/1876 [10:46<19:15,  1.04it/s]
Iteration:  36% 673/1876 [10:47<19:11,  1.04it/s]
Iteration:  36% 674/1876 [10:48<19:11,  1.04it/s]
Iteration:  36% 675/1876 [10:49<19:10,  1.04it/s]
Iteration:  36% 676/1876 [10:50<19:13,  1.04it/s]
Iteration:  36% 677/1876 [10:51<19:14,  1.04it/s]
Iteration:  36% 678/1876 [10:52<19:12,  1.04it/s]
Iteration:  36% 679/1876 [10:53<19:16,  1.03it/s]
Iteration:  36% 680/1876 [10:54<19:15,  1.04it/s]
Iteration:  36% 681/1876 [10:55<19:19,  1.03it/s]
Iteration:  36% 682/1876 [10:56<19:20,  1.03it/s]
Iteration:  36% 683/1876 [10:57<19:13,  1.03it/s]
Iteration:  36% 684/1876 [10:58<19:07,  1.04it/s]
Iteration:  37% 685/1876 [10:59<19:03,  1.04it/s]


In [ ]:
!python evaluation.py --task_name sentihood_QA_M --pred_data_dir results/test_ep_4.txt

aspect_strict_Acc = 0.7035657264502395
aspect_Macro_F1 = 0.6950111090269153
aspect_Macro_AUC = 0.9535790668161686
sentiment_Acc = 0.890625
sentiment_Macro_AUC = 0.9406718510981621
